<a href="https://colab.research.google.com/github/elizathornton/REACH_NP/blob/main/NP_Order_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries

!pip install jupyter-dash
!pip install dash-bootstrap-components

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


from jupyter_dash import JupyterDash
import dash
from dash import Dash, dcc, html, Input, Output, State
from dash.html import Div
import dash_bootstrap_components as dbc

from google.colab import drive
drive.mount('/content/drive')






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 1.1 MB/s 
     |████████████████████████████████| 357 kB 39.0 MB/s 
     |████████████████████████████████| 1.6 MB 45.6 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=18a65f9fa9137bb4f8e28f3cc99341bbc4f05e36aa934600c5fa8f7567c5c985
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.2 MB/s 
Mounted at /content/drive


In [ ]:
# npdata dataframe has 3 columns:  NP name,  Supervising Doc, State
# reachdata is a one hot encoded dataframe with the index Order type and columns State 0 = Doc sign 1 = NP sign
# nonreachdata is a one hot encoded dataframe with the index Order type and columns State 0 = Doc sign 1 = NP sign


npdata = pd.read_csv('/content/drive/My Drive/VPA Projects/REACH/NP_Order_Guide_NP_List.csv',index_col=0)
reachdata = pd.read_csv('/content/drive/My Drive/VPA Projects/REACH/NP_Order_Guide_REACH_Rules.csv', index_col=0)
nonreachdata = pd.read_csv('/content/drive/My Drive/VPA Projects/REACH/NP_Order_Guide_Non_REACH_Rules.csv', index_col=0)

#reachdata = reachdata.set_index('Order Type')

npdata.head()

,Supervising Doc,State
NP,,
"Adler, Marthe","Thornton, Eliza",PA
"Alex, Joji",Not Sure Which Doc,WA
"Alonso-Slusarz, Judy","Grey, Shahrzad",FL
"Asare, Louisa",Not Sure Which Doc,TX
"Baessler, Matthew","Rahman, Salman",OH


In [ ]:
nplist = npdata['NP'].tolist()
typelist = reachdata.index.tolist()


In [ ]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = ['https://drive.google.com/uc?export=view&id=1X6bPm9FF9j4udNkpVvtS7sokh_7VIJRhZdD030nTeX0']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
            html.H1('Who should sign the orders?', style ={ 'text-align':'center', 'padding': '0px 20px 20px 20px'}),
            html.Div([
                html.Div('Is this a REACH patient?',style={ 'display': 'inline-block'}),
                dcc.RadioItems(
                  id='reach',
                  options=[{'label': i, 'value': i} for i in ['REACH Patient', 'NOT REACH Patient']],
                  value='NOT REACH Patient',
                  labelStyle={'display': 'block'}
              ),],
              style={ 'display': 'inline-block'}
            ),
            html.Div([
              html.Div('Select the NP:   ', style = {'display': 'inline-block'}),
              dcc.Dropdown(
                  id='npprovider',
                  options=[{'label': i, 'value': i} for i in nplist],
 #                 inline = True,
 #                 style ={  'padding': '0px 20px 20px 20px'},
                 style ={'width':'50%','display':'inline-block'}
            )]),
            html.Div([
              html.Div('Select the Order Type:   ', style = {'display': 'inline-block'}),
              dcc.Dropdown(
                id='doctype',
                options=[{'label': i, 'value': i} for i in typelist],
                style ={'width':'50%', 'display':'inline-block'}
            ),
            html.H1(
                id = 'sendto',
                style ={ 'text-align':'center', 'padding': '20px 20px 20px 20px'})
            ])
])

@app.callback(
    dash.dependencies.Output('sendto', 'children'),
    [dash.dependencies.Input('npprovider', 'value'),
     dash.dependencies.Input('doctype', 'value'),
     dash.dependencies.Input('reach', 'value')])


def whotosendto(npprovider, doctype, reach):
   
   # supervisingdoc = npdata[npdata['NP'] == npprovider]['Supervising Doc'].values[0]
    supervisingdoc = npdata.loc[npprovider, 'Supervising Doc']
    whichstate = npdata.loc[npprovider, 'State']


    if reach == 'REACH Patient':
      whosigns = reachdata.loc[doctype, whichstate]
    else:
      whosigns = nonreachdata.loc[doctype, whichstate]
  
    if whosigns == 1:
      return 'The NP should sign the order.  Send to: ' + npprovider + '.'
    elif whosigns == 0:
      return 'The Doc should sign the order.  Send to: ' + supervisingdoc + '.'







In [ ]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>